# <font color = 'royalblue'> <center> Homework: *State Your Assumptions*

### Get the Data

In [1]:
# load packages
from dvc.api import read,get_url
import pandas as pd
import re

# store text file
txt = read('resources/data/shakespeare/shakespeare.txt', 
           repo='https://github.com/TLP-COI/text-data-course')

# view
print(txt[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Part 1

Split the text file into a table, such that

- each row is a single line of dialogue
- there are columns for
    - the speaker
    - the line number
    - the line dialogue (the text)

Hint: you will need to use RegEx to do this rapidly. See the in-class "markdown" example!

Question(s):

What assumptions have you made about the text that allowed you to do this?

In [189]:
# pattern to store speakers
patt_speaker = re.compile(
    "([A-Z][\w-]*(\s[A-Z][\w-]*)*)[$:][\r\n]",
    flags=re.S | re.M
)

# find matches
speaker_matches = patt_speaker.findall(txt)

# store in temporary dataframe
speaker_df = pd.DataFrame.from_records(speaker_matches, columns=['speaker', 'temp'])

# view
speaker_df.speaker[:10]

0     First Citizen
1               All
2     First Citizen
3               All
4     First Citizen
5               All
6     First Citizen
7               All
8    Second Citizen
9     First Citizen
Name: speaker, dtype: object

In [196]:
### this is not working 

# store pattern for dialogues
patt_dialogue = re.compile(
    "^[^\r\n:-][^\r\n:]*:\s*$" 
    "(.*?)"
    "(?=^(^[^\r\n:-][^\r\n:]*:\s*$)|\Z)", 
    flags = re.S | re.M
)

# find matches
dialogue_matches = patt_dialogue.findall(txt)

# store in temporary dataframe
dialogues_df = pd.DataFrame.from_records(dialogue_matches, columns=['dialogue', 'speaker'])

# check
dialogues_df[25:31]

,dialogue,speaker
25,"\nWe cannot, sir, we are undone already.\n\n",MENENIUS:
26,"\nI tell you, friends, most charitable care\nH...",First Citizen:
27,"\nCare for us! True, indeed! They ne'er cared ...",MENENIUS:
28,\nEither you must\nConfess yourselves wondrous...,First Citizen:
29,"\nWell, I'll hear it, sir: yet you must not th...",MENENIUS:
30,\nThere was a time when all the body's members\n,"Rebell'd against the belly, thus accused it:"


In [ ]:
re_newline = re.compile(r'\n')

# create list to store line numbers
line_number = []

# calculate and store line numbers of dialogues
for m in patt_dialogue.finditer(txt):
    start_line = len(re_newline.findall(txt, 0, m.start(1)))+1
    line_number.append(start_line)